## Este código limpia y acota las bases de datos del código de sector 46 del NAICS

Este código está tratando de unir los datos de los negocios de tipo *comercio al por menor*. Asimismo, la base final de este código es una base distinta de la cual se está obteniendo los resultados principales. Esta base tiene el objetivo de construir una parte de la variable instrumental que permitirá explicar por qué Airbnb pudo haber reducido el crimen.

In [12]:
import pandas as pd
import os

In [13]:
#Definimos la carpeta en donde están los datos
carpeta = r"C:\Users\santo\Documents\CIDE\Tesis\datos_negocios"

In [14]:
#Columnas que ocuparé
columnas_deseadas = ['id', 'latitud', 'longitud', 'cve_ent']

archivos = sorted([f for f in os.listdir(carpeta) if f.startswith("denue_inegi_46") and f.endswith(".csv")])

dataframe = []

for archivo in archivos:
    ruta = os.path.join(carpeta, archivo)
    
    # Extraer año y mes desde el nombre (e.g. 'denue_inegi_46_2019_04_2.csv')
    partes = archivo.replace(".csv", "").split("_")
    year, mes = partes[3], partes[4]
    
    if mes == "04":
        mes = "03"
    elif mes == "11":
        mes = "09"#este cambio de meses se hizo porque el periodo del análisis final del trabajo es de 2019-03 a 2020-03. Se buscaba tener información para el inicio y la mitad del periodo
    
    fecha = pd.to_datetime(f"{year}-{mes}").to_period("M")
    
    df = (pd.read_csv(ruta, usecols=columnas_deseadas, encoding="latin-1")
          .query("cve_ent == 9")#dayos solo de la CDMX. Ese es el código de la CDMX.
          .assign(mes=fecha)
    )
    dataframe.append(df)

df_concatenado = pd.concat(dataframe, ignore_index=True)
df_concatenado

,id,cve_ent,latitud,longitud,mes
0,668420,9,19.475395,-99.176682,2019-03
1,936901,9,19.262586,-99.087953,2019-03
2,708545,9,19.391953,-99.089058,2019-03
3,1017937,9,19.381628,-99.235162,2019-03
4,910897,9,19.320018,-99.154361,2019-03
...,...,...,...,...,...
611284,8277614,9,19.473872,-99.081538,2020-03
611285,867034,9,19.409391,-99.176411,2020-03
611286,775801,9,19.398283,-99.184954,2020-03
611287,8842074,9,19.414390,-99.144841,2020-03


## Preparamos la base para exportarla

In [15]:
df_final = df_concatenado.drop(columns=['cve_ent'])

carpeta_final = r"C:\Users\santo\Documents\CIDE\Tesis\datos_unidos"

df_final.to_csv(os.path.join(carpeta_final, "negocios_limpio.csv"), index=False, encoding='utf-8-sig')